In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
import csv
 
#원하는 페이지 설정(변경 가능)
url = 'https://play.google.com/store/apps/details?id=com.omnitel.android.lottewebview&amp;showAllReviews=true'
short = 1
long = 3
 
#ChromeDriver 위치(변경 필요)
driver = webdriver.Chrome(executable_path = 'C:\\Users\\gorgo\\Desktop\\chromedriver_win32\\chromedriver.exe')
driver.get(url)
driver.implicitly_wait(long)
driver.maximize_window()
driver.implicitly_wait(short)
listbox = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/c-wiz/div[1]/div')
listbox.click()
driver.implicitly_wait(short)
select = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/c-wiz/div[1]/div/div[2]/div[1]')
select.click()
driver.implicitly_wait(short)
 
#스크롤바 첫 버튼 발견할 때까지 내림
while(True):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.5)
    try:
        #버튼 찾기
        element = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div')
        if(element is not None):
            element.click() #버튼 작동
            break
    except Exception:
        continue
             
errTime = 0
successTime = 0
while(errTime < 10 and successTime < 3): #더보기 버튼을 30번 누를 때 까지 반복
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.5)
    try:
        element = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div')
        if(element is not None):
            element.click() #버튼 작동
            successTime += 1
            errTime = 0
    except Exception:
        errTime += 1

#page source parsing
html = driver.page_source
bs0bj = BeautifulSoup(html, 'lxml')
div_reviews = bs0bj.find_all('div', {'class':'d15Mdf bAhLNe'})
reviews = []
for div in div_reviews:
    try:
        grade = len(div.find_all('div', {'class':'vQHuPe bUWb7c'}))
        date = div.find('span', {"class":'p2TkOb'}).get_text()
        t = re.findall(r"\d*\.\d+|\d+", date)
        date = '{0}-{1}-{2}'.format(t[0], t[1], t[2])
        good = div.find('div', {"class": "jUL89d y92BAb"}).get_text()
        content = div.find('span', {"jsname":"bN97Pc"}).get_text()
        content = content.replace("전체 리뷰", "")
        content = re.sub('[^가-힝0-9a-zA-Z_!?@#%^&-=:;,\"\'<>\\s]', '', content)
        content.encode('euc-kr')
        reviews.append((content, grade, good, date))
    except Exception:
        continue

#저장
with open("review.csv", 'wt', encoding = 'euc-kr', newline = '')  as file:
    csvWriter = csv.writer(file)
    csvWriter.writerow(["내용","평점","공감수","작성일"])
    for rev in reviews:
        csvWriter.writerow(rev)